In [2]:
# RUNNING ONLY
# import library
import psycopg2
import numpy as np
import pandas as pd
from getpass import getpass

In [3]:
# REQUIRE INPUT
# fill in username/ password
user = 'lixan23'
db_ip = '10.158.72.112'
################################
pw = getpass('Enter the database password for {}: '.format(user))

Enter the database password for lixan23: ········


In [4]:
# RUNNING ONLY
# run this train set creation function - still include all feature if later require statistics
# redundant features will be filter in data process function below
# dont drop customer_id this time
def get_dataset( reference_day, tumbling_window_size = 38, output_window_size = 38 ):
    with psycopg2.connect("host='{}' dbname='nlab' user='{}' password='{}'".format(db_ip, user, pw)) as conn:
      sql = """
      WITH 
      tumbling AS(
      SELECT customer_id,
             %(ref_date)s::date AS ref_day,
             COUNT (DISTINCT CASE WHEN purchased_at > %(ref_date)s::DATE AND purchased_at <= %(ref_date)s::DATE + %(ows)s THEN purchased_at::DATE ELSE null END) as output_feature,
             SUM(CASE WHEN purchased_at > %(ref_date)s::DATE -%(ws)s AND purchased_at <= %(ref_date)s::DATE THEN value ELSE 0 END) as sale_f1,
             SUM(CASE WHEN purchased_at > %(ref_date)s::DATE -%(ws)s*2 AND purchased_at <= %(ref_date)s::DATE-%(ws)s THEN value ELSE 0 END ) as sale_f2,
             SUM(CASE WHEN purchased_at > %(ref_date)s::DATE -%(ws)s*3 AND purchased_at <= %(ref_date)s::DATE-%(ws)s*2 THEN value ELSE 0 END ) as sale_f3,
             SUM(CASE WHEN purchased_at > %(ref_date)s::DATE -%(ws)s*4 AND purchased_at <= %(ref_date)s::DATE-%(ws)s*3 THEN value ELSE 0 END ) as sale_f4,
             SUM(CASE WHEN purchased_at > %(ref_date)s::DATE -%(ws)s*5 AND purchased_at <= %(ref_date)s::DATE-%(ws)s*4 THEN value ELSE 0 END ) as sale_f5,
             COUNT (DISTINCT CASE WHEN purchased_at > %(ref_date)s::DATE -%(ws)s AND purchased_at <= %(ref_date)s::DATE THEN purchased_at::DATE ELSE null END ) as visit_f1,
             COUNT (DISTINCT CASE WHEN purchased_at > %(ref_date)s::DATE -%(ws)s*2 AND purchased_at <= %(ref_date)s::DATE-%(ws)s THEN purchased_at::DATE ELSE null END ) as visit_f2,
             COUNT (DISTINCT CASE WHEN purchased_at > %(ref_date)s::DATE -%(ws)s*3 AND purchased_at <= %(ref_date)s::DATE-%(ws)s*2 THEN purchased_at::DATE ELSE null END ) as visit_f3,
             COUNT (DISTINCT CASE WHEN purchased_at > %(ref_date)s::DATE -%(ws)s*4 AND purchased_at <= %(ref_date)s::DATE-%(ws)s*3 THEN purchased_at::DATE ELSE null END ) as visit_f4,
             COUNT (DISTINCT CASE WHEN purchased_at > %(ref_date)s::DATE -%(ws)s*5 AND purchased_at <= %(ref_date)s::DATE-%(ws)s*4 THEN purchased_at::DATE ELSE null END ) as visit_f5
     FROM ml2.receipts_clean
     JOIN ml2.receipt_lines_clean
     USING (receipt_id)
     GROUP BY customer_id),
     
     rfm AS (
     SELECT customer_id,
             SUM(value) as total_spend,
             COUNT (DISTINCT purchased_at::DATE) as active_day,
             SUM(value)/COUNT (DISTINCT purchased_at) as basket_value,
             SUM (value)/ SUM (qty) as unit_cost,
             COUNT (DISTINCT product_code) as num_pro,
             COUNT(DISTINCT purchased_at::DATE)/((EXTRACT(day from (MAX(purchased_at)-MIN(purchased_at)))/%(ws)s)::INT+1) AS average_visit,
             SUM (value)/((EXTRACT(day from (MAX(purchased_at)-MIN(purchased_at)))/%(ws)s)::INT+1) AS average_spend
     FROM ml2.receipts_clean
     JOIN ml2.receipt_lines_clean
     USING (receipt_id)
     --below condition to make sure we do not include future data--
     WHERE purchased_at<=%(ref_date)s::date
     --only consider the nearest 5 period
     AND purchased_at>%(ref_date)s::DATE -%(ws)s*5
     GROUP BY customer_id
     HAVING MAX(purchased_at)>%(ref_date)s::DATE-%(ws)s),
             
     --create static value as most frequently visit store--
     store AS (
     SELECT customer_id, store_code 
            FROM (SELECT customer_id, store_code, COUNT(*) AS ct
                    FROM ml2.receipts_clean
                    GROUP BY 1,2)x
            GROUP BY 1,2,ct
            HAVING ct=MAX(ct))
    
    SELECT * FROM tumbling JOIN store USING (customer_id) JOIN rfm USING (customer_id)
    
      """

      df = pd.read_sql(sql, conn, params = {'ref_date':reference_day, 'ws':tumbling_window_size, 'ows':output_window_size})

      return df.drop(columns = ['ref_day','output_feature'], inplace = False), df.output_feature

In [5]:
# RUNNING ONLY
# create processing function includes:
def data_process (X,y):
    for i in range(len(y)):
        # process output to churn/ non churn based on decreasethreshold = 0.49
        if y[i]<=X.iloc[i,-2]*0.49:
            y[i]=1
        else:
            y[i]=0
    # one hot coding process for store inputs
    from sklearn.preprocessing import LabelBinarizer
    lb = LabelBinarizer()
    def binary_convert(df,a):
        lb_results = lb.fit_transform(df[a])
        df_onehot = pd.DataFrame(lb_results, columns=lb.classes_)
        for col in list(df_onehot.columns):
            df_onehot.rename(columns={col:(a+'_'+str(col))},inplace = True)
        df=df.drop(columns=a)
        new_df=pd.concat([df,df_onehot],axis=1)
        return new_df
    X_new=binary_convert(X,'store_code')
    # define X for prediction
    X_min=X_new[['visit_f1','visit_f2','visit_f3','visit_f4','visit_f5','active_day','basket_value','unit_cost','num_pro','average_visit']]
    return X_new, X_min, y

In [6]:
# REQUIRE INPUT
import datetime  
from datetime import timedelta
import numpy as np
# please input the latest date here - for now running the lastest date from database
today= datetime.date(2020,11,24)

In [7]:
# RUNNING ONLY
# getting data
# training ref day will be today - 76 days
ref_train= str(today-timedelta(days=38))
X_train, y_train  = get_dataset(ref_train)
X_train, X_train_min, y_train=data_process (X_train,y_train)
# test ref day will be today - 38 days
# y_test will receive 0 value as we're at observation period
ref_test= str(today)
X_test, y_test  = get_dataset(ref_test)
X_test, X_test_min, y_test=data_process (X_test,y_test)

In [8]:
# RUNNING ONLY
# fitting model
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(random_state=42, max_depth=3, n_estimators=50)
rf.fit(X_train_min,y_train)
# merging data with prediction
predict_percent=pd.DataFrame(rf.predict_proba(X_test_min))[1]
predict_label=pd.DataFrame(rf.predict(X_test_min))
df=pd.concat([X_test,predict_percent,predict_label],axis=1)
# Rename the columns
df.rename(columns={1:'churn_percent'},inplace=True)
df.rename(columns={0:'churn_label'},inplace=True)
# Rename the prediction result to churn - non churn
for i in range(len(df)):
    if df['churn_label'][i]==1:
        df['churn_label'][i]='Churn'
    else:
        df['churn_label'][i]='Non_churn'

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [9]:
# RUNNING ONLY
# Create ranking columns based on prediction probability
# 1: >=90# ... 10:<10%
df['ranking']=((1-df['churn_percent'])*100)//10+1

In [10]:
# RUNNING ONLY
# generate list of customer to be targeted
# list contain people to be targeted at top
df[['customer_id','total_spend','churn_percent','ranking']].sort_values(by='ranking')

,customer_id,total_spend,churn_percent,ranking
597,10920,3.40,0.905592,1.0
558,10343,4.33,0.905592,1.0
165,3193,3.83,0.900027,1.0
202,3822,7.24,0.901663,1.0
729,13542,4.49,0.900027,1.0
...,...,...,...,...
457,8454,3104.61,0.058311,10.0
115,2311,1283.59,0.053367,10.0
466,8723,828.62,0.062239,10.0
760,14131,1569.81,0.068287,10.0


In [11]:
# RUNNING ONLY
# Calculating % of each group - choosing relevant group to target based on their % and active day
# People with 1 active day - First time group
# People with >=2 - Rising interest group
for c, d in df.groupby('ranking'):
    m=np.mean(d['active_day'])
    print ('- The target group {} with churn probability around {}% contains {} people, \naccounts for {}% of active base, has {} active days'.format(c, 100-(c+1)*10,len(d),round(len(d)*100/len(df),1),round(m,1)))

- The target group 1.0 with churn probability around 80.0% contains 26 people, 
accounts for 2.9% of active base, has 1.0 active days
- The target group 2.0 with churn probability around 70.0% contains 123 people, 
accounts for 13.9% of active base, has 1.1 active days
- The target group 3.0 with churn probability around 60.0% contains 50 people, 
accounts for 5.6% of active base, has 1.8 active days
- The target group 4.0 with churn probability around 50.0% contains 63 people, 
accounts for 7.1% of active base, has 2.1 active days
- The target group 5.0 with churn probability around 40.0% contains 38 people, 
accounts for 4.3% of active base, has 3.3 active days
- The target group 6.0 with churn probability around 30.0% contains 61 people, 
accounts for 6.9% of active base, has 4.2 active days
- The target group 7.0 with churn probability around 20.0% contains 83 people, 
accounts for 9.4% of active base, has 5.4 active days
- The target group 8.0 with churn probability around 10.0% c

In [12]:
# RUNNING ONLY
# Full data table by group
pd.set_option('display.max_columns', None)
for c, d in df.groupby('ranking'):
    print (c)
    print('% size ',len(d)/len(df)*100)
    print ('% sale: ',np.sum(d['total_spend'])/np.sum(df['total_spend'])*100)
    display(d.describe())

1.0
% size  2.937853107344633
% sale:  0.050019621280010314


,customer_id,sale_f1,sale_f2,sale_f3,sale_f4,sale_f5,visit_f1,visit_f2,visit_f3,visit_f4,visit_f5,total_spend,active_day,basket_value,unit_cost,num_pro,average_visit,average_spend,store_code_0,store_code_1,store_code_2,store_code_3,churn_percent,ranking
count,26.000000,26.000000,26.0,26.0,26.0,26.0,26.0,26.0,26.0,26.0,26.0,26.000000,26.0,26.000000,26.000000,26.000000,26.0,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.0
mean,9236.423077,4.598462,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.598462,1.0,4.598462,3.400513,1.307692,1.0,4.598462,0.192308,0.115385,0.423077,0.269231,0.903610,1.0
std,4726.331788,1.042653,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.042653,0.0,1.042653,1.034003,0.549125,0.0,1.042653,0.401918,0.325813,0.503831,0.452344,0.002612,0.0
min,148.000000,2.850000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.850000,1.0,2.850000,1.915000,1.000000,1.0,2.850000,0.000000,0.000000,0.000000,0.000000,0.900027,1.0
25%,5270.750000,3.925000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.925000,1.0,3.925000,2.547500,1.000000,1.0,3.925000,0.000000,0.000000,0.000000,0.000000,0.900027,1.0
50%,10631.500000,4.345000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4.345000,1.0,4.345000,3.400000,1.000000,1.0,4.345000,0.000000,0.000000,0.000000,0.000000,0.905592,1.0
75%,13256.000000,5.105000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.105000,1.0,5.105000,4.250000,1.750000,1.0,5.105000,0.000000,0.000000,1.000000,0.750000,0.905592,1.0
max,15889.000000,7.240000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,7.240000,1.0,7.240000,5.860000,3.000000,1.0,7.240000,1.000000,1.000000,1.000000,1.000000,0.905592,1.0


2.0
% size  13.898305084745763
% sale:  0.9871804848171456


,customer_id,sale_f1,sale_f2,sale_f3,sale_f4,sale_f5,visit_f1,visit_f2,visit_f3,visit_f4,visit_f5,total_spend,active_day,basket_value,unit_cost,num_pro,average_visit,average_spend,store_code_0,store_code_1,store_code_2,store_code_3,churn_percent,ranking
count,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.0,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.000000,123.0
mean,8054.341463,17.562358,0.331870,0.417073,0.329919,0.542683,1.0,0.016260,0.024390,0.040650,0.040650,19.183902,1.121951,17.628699,5.910035,3.162602,0.902439,16.994068,0.252033,0.146341,0.308943,0.292683,0.874612,2.0
std,4779.543047,19.939529,3.065201,2.800516,1.759873,2.974345,0.0,0.126992,0.154888,0.198287,0.198287,20.492453,0.328568,19.898075,12.376649,1.865889,0.297934,20.098188,0.435956,0.354894,0.463947,0.456855,0.030977,0.0
min,22.000000,1.190000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,1.190000,1.000000,1.190000,0.500000,1.000000,0.000000,1.017500,0.000000,0.000000,0.000000,0.000000,0.800008,2.0
25%,3482.500000,8.235000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,8.740000,1.000000,8.275000,2.288095,2.000000,1.000000,7.645000,0.000000,0.000000,0.000000,0.000000,0.872931,2.0
50%,8439.000000,12.350000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,13.340000,1.000000,12.350000,3.410000,3.000000,1.000000,11.980000,0.000000,0.000000,0.000000,0.000000,0.889021,2.0
75%,12094.500000,20.660000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,21.695000,1.000000,20.465000,5.709167,4.000000,1.000000,19.540000,0.500000,0.000000,1.000000,1.000000,0.897036,2.0
max,16229.000000,144.040000,33.190000,23.310000,13.140000,23.870000,1.0,1.000000,1.000000,1.000000,1.000000,144.040000,2.000000,144.040000,127.660000,11.000000,1.000000,144.040000,1.000000,1.000000,1.000000,1.000000,0.898586,2.0


3.0
% size  5.649717514124294
% sale:  0.6400428070228285


...

,customer_id,sale_f1,sale_f2,sale_f3,sale_f4,sale_f5,visit_f1,visit_f2,visit_f3,visit_f4,visit_f5,total_spend,active_day,basket_value,unit_cost,num_pro,average_visit,average_spend,store_code_0,store_code_1,store_code_2,store_code_3,churn_percent,ranking
count,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.000000,193.0
mean,8048.678756,50.054456,30.231295,35.235699,33.698860,40.250622,3.025907,1.917098,2.269430,1.911917,2.357513,189.470933,11.481865,16.534583,1.640724,64.331606,1.953368,38.483157,0.222798,0.124352,0.367876,0.284974,0.247318,8.0
std,4685.957133,55.063762,38.244033,44.211956,53.114238,57.740217,2.204246,1.724024,2.160234,2.076032,3.031374,166.791378,6.702839,11.546006,1.092054,43.004771,1.433564,33.528732,0.417206,0.330841,0.483481,0.452576,0.028270,0.0
min,21.000000,1.310000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,29.480000,4.000000,3.238182,0.712357,5.000000,1.000000,7.351667,0.000000,0.000000,0.000000,0.000000,0.200064,8.0
25%,4039.000000,16.400000,7.380000,5.060000,0.000000,3.900000,2.000000,1.000000,1.000000,0.000000,1.000000,95.490000,7.000000,8.974286,1.068611,35.000000,1.000000,17.433333,0.000000,0.000000,0.000000,0.000000,0.220834,8.0
50%,8283.000000,31.150000,19.760000,23.000000,14.640000,23.830000,2.000000,2.000000,2.000000,1.000000,2.000000,126.300000,10.000000,13.871000,1.278348,52.000000,2.000000,27.020000,0.000000,0.000000,0.000000,0.000000,0.246148,8.0
75%,11816.000000,64.910000,36.510000,46.490000,39.770000,50.330000,4.000000,2.000000,3.000000,3.000000,3.000000,221.640000,14.000000,20.403750,1.664539,85.000000,2.000000,45.110000,0.000000,0.000000,1.000000,1.000000,0.271044,8.0
max,16259.000000,372.290000,222.850000,270.140000,328.360000,375.830000,18.000000,10.000000,11.000000,13.000000,21.000000,848.250000,56.000000,82.090000,9.815897,214.000000,11.000000,214.150000,1.000000,1.000000,1.000000,1.000000,0.299854,8.0


9.0
% size  20.0
% sale:  31.612308608847066


,customer_id,sale_f1,sale_f2,sale_f3,sale_f4,sale_f5,visit_f1,visit_f2,visit_f3,visit_f4,visit_f5,total_spend,active_day,basket_value,unit_cost,num_pro,average_visit,average_spend,store_code_0,store_code_1,store_code_2,store_code_3,churn_percent,ranking
count,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.000000,177.0
mean,8128.768362,87.414746,99.139209,93.467232,75.518701,71.362373,5.248588,5.813559,5.548023,4.553672,4.502825,426.902260,25.666667,17.092036,1.483698,112.163842,4.384181,79.375687,0.192090,0.101695,0.395480,0.310734,0.163390,9.0
std,4790.278857,68.659080,82.754316,92.957569,72.520234,68.627850,2.849504,3.145024,3.818199,3.717083,3.842236,295.850166,12.872039,8.698290,0.670059,63.032200,2.139822,54.187034,0.395061,0.303104,0.490341,0.464107,0.027475,0.0
min,65.000000,7.720000,18.820000,0.000000,0.000000,0.000000,2.000000,2.000000,0.000000,0.000000,0.000000,45.480000,6.000000,5.834286,0.749713,16.000000,1.000000,20.595000,0.000000,0.000000,0.000000,0.000000,0.104143,9.0
25%,4227.000000,41.070000,47.810000,39.970000,24.450000,21.210000,3.000000,3.000000,3.000000,2.000000,2.000000,220.260000,16.000000,11.143684,1.057216,65.000000,3.000000,43.998000,0.000000,0.000000,0.000000,0.000000,0.141631,9.0
50%,7007.000000,69.830000,72.250000,66.380000,54.550000,57.190000,5.000000,5.000000,5.000000,4.000000,4.000000,340.480000,23.000000,15.209167,1.240034,100.000000,4.000000,65.662000,0.000000,0.000000,0.000000,0.000000,0.171413,9.0
75%,12616.000000,118.220000,124.240000,125.930000,104.670000,111.420000,7.000000,7.000000,8.000000,7.000000,7.000000,510.130000,33.000000,20.852333,1.705349,146.000000,6.000000,94.442000,0.000000,0.000000,1.000000,1.000000,0.186417,9.0
max,16275.000000,460.340000,414.570000,759.650000,346.670000,359.950000,18.000000,17.000000,20.000000,18.000000,26.000000,1795.990000,72.000000,54.423939,4.596780,349.000000,12.000000,448.997500,1.000000,1.000000,1.000000,1.000000,0.198678,9.0


10.0
% size  8.022598870056497
% sale:  44.69266967386839


,customer_id,sale_f1,sale_f2,sale_f3,sale_f4,sale_f5,visit_f1,visit_f2,visit_f3,visit_f4,visit_f5,total_spend,active_day,basket_value,unit_cost,num_pro,average_visit,average_spend,store_code_0,store_code_1,store_code_2,store_code_3,churn_percent,ranking
count,71.000000,71.000000,71.000000,71.000000,71.000000,71.00000,71.000000,71.000000,71.000000,71.000000,71.000000,71.00000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.000000,71.0
mean,8751.718310,325.702958,317.711127,311.104930,274.799296,275.29000,18.042254,16.591549,16.718310,15.112676,13.732394,1504.60831,80.197183,18.836738,1.500063,176.225352,13.070423,251.672376,0.267606,0.140845,0.408451,0.183099,0.065894,10.0
std,4646.297922,189.913567,207.368718,230.148324,190.809134,194.19165,7.102186,7.961654,7.572852,7.407620,6.938418,920.02960,31.712827,9.033634,0.566496,75.789407,5.221724,152.549554,0.445862,0.350338,0.495046,0.389500,0.012839,0.0
min,574.000000,86.380000,30.300000,49.860000,6.450000,14.23000,8.000000,4.000000,5.000000,1.000000,1.000000,379.82000,36.000000,8.224853,0.784734,47.000000,6.000000,63.303333,0.000000,0.000000,0.000000,0.000000,0.051746,10.0
25%,4522.500000,203.590000,182.075000,177.060000,138.160000,131.85000,12.000000,11.000000,10.000000,10.000000,9.500000,866.46500,58.500000,12.571974,1.132055,118.500000,10.000000,145.306667,0.000000,0.000000,0.000000,0.000000,0.055932,10.0
50%,8454.000000,256.250000,248.760000,260.140000,235.340000,220.89000,16.000000,15.000000,15.000000,14.000000,13.000000,1283.59000,73.000000,16.449811,1.292970,173.000000,12.000000,213.931667,0.000000,0.000000,0.000000,0.000000,0.060141,10.0
75%,13192.500000,401.840000,394.580000,351.715000,374.530000,390.77000,20.500000,21.000000,23.000000,19.000000,16.000000,1866.31000,92.500000,22.232579,1.781413,213.000000,15.000000,311.051667,1.000000,0.000000,1.000000,0.000000,0.069744,10.0
max,16313.000000,966.200000,881.310000,1235.110000,890.130000,770.41000,35.000000,36.000000,35.000000,33.000000,31.000000,4645.03000,167.000000,46.919495,3.177187,463.000000,27.000000,774.171667,1.000000,1.000000,1.000000,1.000000,0.098565,10.0


In [14]:
# REQUIRE INPUT
# create list of offer taker people to monitor
offer=[0]*len(df)
df['offer']=offer
for i in range(len(df)):
    if df['ranking'][i]<=2: #please input target group, assume currently target 1,2
        df['offer'][i]+=1
df[['customer_id','total_spend','churn_percent','ranking','offer']]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,customer_id,total_spend,churn_percent,ranking,offer
0,21,465.50,0.238146,8.0,0
1,22,17.75,0.894897,2.0,1
2,43,100.20,0.329022,7.0,0
3,48,131.51,0.510394,5.0,0
4,50,46.55,0.773408,3.0,0
...,...,...,...,...,...
880,16266,652.17,0.174744,9.0,0
881,16275,131.72,0.186371,9.0,0
882,16291,34.47,0.311735,7.0,0
883,16313,1611.70,0.057371,10.0,0
